In [20]:
from dask.distributed import Client, progress
client = Client()
client

C:\Users\Sebastian\AppData\Local\conda\conda\envs\spark\lib\site-packages\distributed\bokeh\core.py:56: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:63246 Dashboard: http://127.0.0.1:63249/status,Cluster Workers: 4 Cores: 4 Memory: 7.47 GB


In [21]:
import dask.dataframe as dd

In [22]:
df = dd.read_csv('Road_Weather_Information_Stations.csv', blocksize = 50e6)

In [23]:
df.head()

,StationName,StationLocation,DateTime,RecordId,RoadSurfaceTemperature,AirTemperature
0,35thAveSW_SWMyrtleSt,"(47.53918, -122.37658)",03/03/2014 12:42:00 PM,672560,53.88,53.88
1,35thAveSW_SWMyrtleSt,"(47.53918, -122.37658)",03/03/2014 12:43:00 PM,672561,54.05,54.05
2,35thAveSW_SWMyrtleSt,"(47.53918, -122.37658)",03/03/2014 12:44:00 PM,672562,54.21,54.21
3,35thAveSW_SWMyrtleSt,"(47.53918, -122.37658)",03/03/2014 12:45:00 PM,672563,54.38,54.38
4,35thAveSW_SWMyrtleSt,"(47.53918, -122.37658)",03/03/2014 12:46:00 PM,672564,54.54,54.54


In [24]:
del df['StationLocation']

In [25]:
del df['RoadSurfaceTemperature']

In [26]:
del df['RecordId']

In [27]:
df.head()

,StationName,DateTime,AirTemperature
0,35thAveSW_SWMyrtleSt,03/03/2014 12:42:00 PM,53.88
1,35thAveSW_SWMyrtleSt,03/03/2014 12:43:00 PM,54.05
2,35thAveSW_SWMyrtleSt,03/03/2014 12:44:00 PM,54.21
3,35thAveSW_SWMyrtleSt,03/03/2014 12:45:00 PM,54.38
4,35thAveSW_SWMyrtleSt,03/03/2014 12:46:00 PM,54.54


In [28]:
def extractDate(df, col):
    df[col] = dd.to_datetime(df[col])
    df['Year'] = df[col].dt.year
    df['Month'] = df[col].dt.month
    df['Day'] = df[col].dt.day

In [29]:
def filterMonths(month):
    if (month == 12) or (month == 1) or (month == 2):
        season = 'WINTER'
    elif (month == 3) or (month == 4) or (month == 5):
        season = 'SPRING'
    elif(month == 6) or (month == 7) or (month == 8):
        season = 'SUMMER'
    else:
        season = 'FALL'
    return season

def createSeasonColumn(df):
    df['Season'] = df.Month.apply(filterMonths)

In [30]:
extractDate(df, 'DateTime')

In [31]:
createSeasonColumn(df)

C:\Users\Sebastian\AppData\Local\conda\conda\envs\spark\lib\site-packages\dask\dataframe\core.py:2252: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [32]:
del df['DateTime']

In [33]:
df.head()

,StationName,AirTemperature,Year,Month,Day,Season
0,35thAveSW_SWMyrtleSt,53.88,2014,3,3,SPRING
1,35thAveSW_SWMyrtleSt,54.05,2014,3,3,SPRING
2,35thAveSW_SWMyrtleSt,54.21,2014,3,3,SPRING
3,35thAveSW_SWMyrtleSt,54.38,2014,3,3,SPRING
4,35thAveSW_SWMyrtleSt,54.54,2014,3,3,SPRING


In [34]:
def filterPrecincts(StationName):
    if (StationName == 'JoseRizalBridgeNorth'):
        Precinct = 'EAST'
    elif (StationName == 'AuroraBridge') or (StationName == 'NE45StViaduct') or (StationName == 'RooseveltWay_NE80thSt'):
        Precinct = 'NORTH'
    elif(StationName == 'AlbroPlaceAirportWay') or (StationName == 'SpokaneSwingBridge'):
        Precinct = 'SOUTH'
    elif(StationName == '35thAveSW_SWMyrtleSt') or (StationName == 'HarborAveUpperNorthBridge'):
        Precinct = 'SOUTHWEST'
    else:
        Precinct = 'SOUTH'
    return Precinct

def createPrecinctColumn(df):
    df['Precinct'] = df.StationName.apply(filterPrecincts)

In [35]:
createPrecinctColumn(df)

C:\Users\Sebastian\AppData\Local\conda\conda\envs\spark\lib\site-packages\dask\dataframe\core.py:2252: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [36]:
del df['StationName']

In [37]:
removeOutliers(df, 'AirTemperature')

,AirTemperature,Year,Month,Day,Season,Precinct
npartitions=57,,,,,,
,float64,int64,int64,int64,object,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [38]:
df.head()

,AirTemperature,Year,Month,Day,Season,Precinct
0,53.88,2014,3,3,SPRING,SOUTHWEST
1,54.05,2014,3,3,SPRING,SOUTHWEST
2,54.21,2014,3,3,SPRING,SOUTHWEST
3,54.38,2014,3,3,SPRING,SOUTHWEST
4,54.54,2014,3,3,SPRING,SOUTHWEST


In [39]:
#df.to_csv('nombre')